In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import pygame
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

class JujutsuKaisenGestureRecognition:
    def __init__(self):
        # Initialize MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Domain definitions
        self.domain_signs = {
            "infinite_void": "無量空處 (Gojo Satoru)",
            "chimera_shadow": "自閉圓頓裡 (Megumi Fushiguro)",
            "idle_transfiguration": "蓋棺鐵處女 (Mahito)",
            "prison_realm": "平安京 (Kenjaku/Geto)"
        }
        
        # Paths for data storage
        self.data_dir = "jujutsu_data"
        self.model_path = "jujutsu_model.pkl"
        self.sound_dir = "sounds"
        self.image_dir = "images"
        
        # Ensure directories exist
        for directory in [self.data_dir, self.sound_dir, self.image_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Initialize pygame for sound
        pygame.mixer.init()
        self.domain_sound = None
        try:
            self.domain_sound = pygame.mixer.Sound("sounds/domain_expansion.wav")
        except:
            print("Sound file not found. Will continue without sound effects.")
        
        # Load or create model
        self.model = None
        if os.path.exists(self.model_path):
            self.load_model()
        
        # Variables for data collection
        self.collecting_data = False
        self.current_domain = ""
        self.collected_frames = []
        
        # Variables for prediction
        self.last_prediction = None
        self.prediction_confidence = 0
        self.prediction_time = 0
        self.cooldown_time = 3  # seconds between predictions
        
        print("✅ Jujutsu Kaisen Domain Expansion recognition system initialized")

    def extract_hand_features(self, image):
        """Extract hand landmarks from an image"""
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)
        
        # Initialize empty feature vector for 2 hands (21 landmarks per hand, 3 values x,y,z per landmark)
        features = np.zeros(126)
        
        if results.multi_hand_landmarks:
            # Process up to 2 hands
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
                # Fill in the features for this hand
                for i, landmark in enumerate(hand_landmarks.landmark):
                    idx = hand_idx * 63 + i * 3
                    features[idx] = landmark.x
                    features[idx + 1] = landmark.y
                    features[idx + 2] = landmark.z
                
                # Draw landmarks on the image
                self.mp_drawing.draw_landmarks(
                    image, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
        
        return features, image

    def collect_data(self):
        """Run the data collection interface"""
        cv2.namedWindow("Jujutsu Kaisen Data Collector", cv2.WINDOW_NORMAL)
        cap = cv2.VideoCapture(0)
        
        print("\n== DATA COLLECTION MODE ==")
        print("Press keys 1-4 to select a domain to collect data for:")
        print("1: Infinite Void (Gojo)")
        print("2: Chimera Shadow (Megumi)")
        print("3: Idle Transfiguration (Mahito)")
        print("4: Prison Realm (Geto/Kenjaku)")
        print("Press 'R' to start/stop recording frames")
        print("Press 'S' to save the collected data")
        print("Press 'Q' to quit")
        
        domains = list(self.domain_signs.keys())
        domain_counts = {domain: 0 for domain in domains}
        
        # Count existing data
        for domain in domains:
            domain_path = os.path.join(self.data_dir, f"{domain}.npy")
            if os.path.exists(domain_path):
                existing_data = np.load(domain_path)
                domain_counts[domain] = len(existing_data)
                print(f"Found {len(existing_data)} existing samples for {domain}")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip the frame for a selfie-view
            frame = cv2.flip(frame, 1)
            
            # Extract hand features
            features, processed_frame = self.extract_hand_features(frame)
            
            # If we're collecting data, store the features
            if self.collecting_data:
                self.collected_frames.append((features, self.current_domain))
                # Display recording indicator
                cv2.putText(processed_frame, f"Recording: {self.current_domain} ({len(self.collected_frames)})",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Display instructions and status
            cv2.putText(processed_frame, "1-4: Select Domain, R: Record, S: Save, Q: Quit",
                        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Display current data counts
            y_pos = 60
            for i, domain in enumerate(domains):
                count_text = f"{i+1}: {domain} - {domain_counts[domain] + (len(self.collected_frames) if self.current_domain == domain and self.collecting_data else 0)}"
                cv2.putText(processed_frame, count_text, (10, y_pos), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                y_pos += 25
            
            cv2.imshow("Jujutsu Kaisen Data Collector", processed_frame)
            
            # Process key presses
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('r'):
                if self.collecting_data:
                    self.collecting_data = False
                    print(f"Stopped recording. Collected {len(self.collected_frames)} frames for {self.current_domain}")
                elif self.current_domain:
                    self.collecting_data = True
                    print(f"Started recording frames for {self.current_domain}")
                else:
                    print("Please select a domain first (press 1-4)")
            elif key == ord('s'):
                if self.collected_frames:
                    self.save_collected_data()
                    # Update counts
                    domain_counts[self.current_domain] += len(self.collected_frames)
                    self.collected_frames = []
                else:
                    print("No data to save")
            elif key in [ord('1'), ord('2'), ord('3'), ord('4')]:
                domain_idx = int(chr(key)) - 1
                if domain_idx < len(domains):
                    self.current_domain = domains[domain_idx]
                    print(f"Selected domain: {self.current_domain}")
                    if self.collecting_data:
                        self.collecting_data = False
                        print("Recording stopped due to domain change")
        
        cap.release()
        cv2.destroyAllWindows()

    def save_collected_data(self):
        """Save the collected frames to disk"""
        if not self.collected_frames:
            print("No data to save")
            return
        
        domain_path = os.path.join(self.data_dir, f"{self.current_domain}.npy")
        
        # Extract features and labels
        features = np.array([f[0] for f in self.collected_frames])
        labels = np.array([f[1] for f in self.collected_frames])
        
        # Check if we already have data for this domain
        if os.path.exists(domain_path):
            existing_data = np.load(domain_path)
            existing_labels = np.load(domain_path.replace(".npy", "_labels.npy"))
            features = np.vstack((existing_data, features))
            labels = np.hstack((existing_labels, labels))
            print(f"Appending {len(self.collected_frames)} frames to existing data")
        else:
            print(f"Saving {len(self.collected_frames)} frames as new data")
        
        # Save the data
        np.save(domain_path, features)
        np.save(domain_path.replace(".npy", "_labels.npy"), labels)
        
        print(f"✅ Saved {len(features)} samples for {self.current_domain}")
        self.collected_frames = []

    def train_model(self):
        """Train the gesture recognition model"""
        print("\n== TRAINING MODEL ==")
        
        # Check if we have data
        if not os.path.exists(self.data_dir):
            print("❌ No data directory found")
            return
        
        # Collect all data
        features = []
        labels = []
        
        for domain in self.domain_signs.keys():
            domain_path = os.path.join(self.data_dir, f"{domain}.npy")
            if os.path.exists(domain_path):
                domain_features = np.load(domain_path)
                domain_labels = np.load(domain_path.replace(".npy", "_labels.npy"))
                
                features.append(domain_features)
                labels.append(domain_labels)
                print(f"Loaded {len(domain_features)} samples for {domain}")
        
        if not features:
            print("❌ No training data found")
            return
        
        # Combine all data
        X = np.vstack(features)
        y = np.hstack(labels)
        
        print(f"Total dataset: {len(X)} samples")
        
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Train a RandomForest classifier
        print("Training RandomForest classifier...")
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)
        
        # Evaluate the model
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model accuracy: {accuracy:.2f}")
        
        # Save the model
        with open(self.model_path, 'wb') as f:
            pickle.dump(self.model, f)
        
        print(f"✅ Model saved to {self.model_path}")

    def load_model(self):
        """Load a trained model"""
        try:
            with open(self.model_path, 'rb') as f:
                self.model = pickle.load(f)
            print(f"✅ Loaded model from {self.model_path}")
        except:
            print(f"❌ Could not load model from {self.model_path}")

    def start_recognition(self):
        """Start the real-time recognition system"""
        if self.model is None:
            print("❌ No model loaded. Please train a model first.")
            return
        
        print("\n== RECOGNITION MODE ==")
        print("Performing real-time Domain Expansion recognition")
        print("Press 'Q' to quit")
        
        cv2.namedWindow("Jujutsu Kaisen Domain Recognition", cv2.WINDOW_NORMAL)
        cap = cv2.VideoCapture(0)
        
        domain_effects = {domain: self.create_domain_effect(domain) for domain in self.domain_signs.keys()}
        showing_effect = False
        effect_start_time = 0
        effect_duration = 3  # seconds
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Flip the frame for a selfie-view
            frame = cv2.flip(frame, 1)
            orig_frame = frame.copy()
            
            # If we're showing an effect, overlay it
            current_time = time.time()
            if showing_effect and current_time - effect_start_time < effect_duration:
                # Continue showing effect
                domain = self.last_prediction
                effect_frame = domain_effects[domain].copy()
                
                # Calculate opacity based on time (fade in and out)
                elapsed = current_time - effect_start_time
                if elapsed < 0.5:  # Fade in
                    alpha = elapsed / 0.5
                elif elapsed > effect_duration - 0.5:  # Fade out
                    alpha = (effect_duration - elapsed) / 0.5
                else:
                    alpha = 1.0
                
                # Create a combined frame
                h, w = frame.shape[:2]
                effect_frame = cv2.resize(effect_frame, (w, h))
                
                # Overlay with transparency
                frame = cv2.addWeighted(frame, 1 - 0.7 * alpha, effect_frame, 0.7 * alpha, 0)
                
                # Add text
                domain_name = self.domain_signs[domain]
                cv2.putText(frame, f"DOMAIN EXPANSION: {domain_name}", 
                            (int(w/2)-200, h-50), cv2.FONT_HERSHEY_DUPLEX, 
                            1.0, (0, 0, 255), 2)
            else:
                showing_effect = False
                
                # Extract hand features and make a prediction
                features, frame = self.extract_hand_features(frame)
                
                # Only predict if we have a model and it's been enough time since the last prediction
                if current_time - self.prediction_time > self.cooldown_time:
                    # Check if any hand is detected
                    if np.any(features != 0):
                        prediction = self.model.predict([features])[0]
                        
                        # Get prediction probabilities
                        probs = self.model.predict_proba([features])[0]
                        confidence = np.max(probs)
                        
                        # Only consider high confidence predictions
                        if confidence > 0.7:
                            if prediction != self.last_prediction:
                                print(f"Detected: {prediction} ({confidence:.2f})")
                                self.last_prediction = prediction
                                self.prediction_confidence = confidence
                                self.prediction_time = current_time
                                
                                # Trigger effect and sound
                                showing_effect = True
                                effect_start_time = current_time
                                
                                # Play sound if available
                                if self.domain_sound:
                                    self.domain_sound.play()
            
            # Display instructions
            cv2.putText(frame, "Press 'Q' to quit", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
            
            # Show the frame
            cv2.imshow("Jujutsu Kaisen Domain Recognition", frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

    def create_domain_effect(self, domain):
        """Create a visual effect for a domain expansion"""
        # Create a basic effect (could be improved with images)
        width, height = 640, 480
        effect = np.zeros((height, width, 3), dtype=np.uint8)
        
        if domain == "infinite_void":
            # Gojo's blue/purple infinity
            for i in range(20):
                radius = int(i * 15)
                color = (138, 43, 226) if i % 2 == 0 else (65, 105, 225)
                cv2.circle(effect, (width//2, height//2), radius, color, 5)
                
        elif domain == "chimera_shadow":
            # Megumi's shadows
            for i in range(height):
                for j in range(width):
                    if np.random.rand() < 0.1:
                        effect[i, j] = (0, 0, np.random.randint(100, 180))
            # Add some shadow creature silhouettes
            for _ in range(5):
                x = np.random.randint(100, width-100)
                y = np.random.randint(100, height-100)
                size = np.random.randint(30, 70)
                cv2.circle(effect, (x, y), size, (0, 0, 0), -1)
                cv2.circle(effect, (x-10, y-10), 5, (255, 0, 0), -1)  # Red eye
                
        elif domain == "idle_transfiguration":
            # Mahito's body transformation
            # Create a flesh-like texture
            for i in range(height):
                for j in range(width):
                    if np.random.rand() < 0.5:
                        effect[i, j] = (np.random.randint(120, 200), 
                                      np.random.randint(100, 150), 
                                      np.random.randint(130, 170))
            # Add "stitches"
            for _ in range(20):
                x1 = np.random.randint(0, width)
                y1 = np.random.randint(0, height)
                x2 = x1 + np.random.randint(-100, 100)
                y2 = y1 + np.random.randint(-100, 100)
                cv2.line(effect, (x1, y1), (x2, y2), (0, 0, 0), 2)
                
        elif domain == "prison_realm":
            # Geto's cursed spirit manipulation
            # Create a dark background with cursed energy
            effect.fill(20)  # Dark background
            # Add some "cursed spirits"
            for _ in range(30):
                x = np.random.randint(0, width)
                y = np.random.randint(0, height)
                size = np.random.randint(10, 40)
                color = (np.random.randint(100, 200), 
                       np.random.randint(100, 200), 
                       np.random.randint(100, 200))
                cv2.circle(effect, (x, y), size, color, -1)
                cv2.circle(effect, (x, y), size+5, (0, 0, 0), 2)
        
        return effect

    def run(self):
        """Main menu interface"""
        while True:
            print("\n== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==")
            print("1: Collect Training Data")
            print("2: Train Model")
            print("3: Start Recognition")
            print("4: Exit")
            
            choice = input("Select an option: ")
            
            if choice == '1':
                self.collect_data()
            elif choice == '2':
                self.train_model()
            elif choice == '3':
                self.start_recognition()
            elif choice == '4':
                print("Exiting...")
                break
            else:
                print("Invalid choice, please try again")


if __name__ == "__main__":
    jjk = JujutsuKaisenGestureRecognition()
    jjk.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Sound file not found. Will continue without sound effects.
✅ Jujutsu Kaisen Domain Expansion recognition system initialized

== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==
1: Collect Training Data
2: Train Model
3: Start Recognition
4: Exit


Select an option:  1



== DATA COLLECTION MODE ==
Press keys 1-4 to select a domain to collect data for:
1: Infinite Void (Gojo)
2: Chimera Shadow (Megumi)
3: Idle Transfiguration (Mahito)
4: Prison Realm (Geto/Kenjaku)
Press 'R' to start/stop recording frames
Press 'S' to save the collected data
Press 'Q' to quit
Selected domain: infinite_void
Started recording frames for infinite_void
Stopped recording. Collected 1335 frames for infinite_void
Started recording frames for infinite_void
Stopped recording. Collected 1356 frames for infinite_void
Saving 1356 frames as new data
✅ Saved 1356 samples for infinite_void
No data to save

== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==
1: Collect Training Data
2: Train Model
3: Start Recognition
4: Exit
